# Tabula Muris

## 소개

단일 세포 RNASeq 데이터셋을 처음부터 끝까지 분석하는 실습 경험을 제공하기 위해 [Tabula Muris](https://tabula-muris.ds.czbiohub.org/)의 데이터를 예시로 사용합니다. Tabula Muris는 모든 마우스 조직을 단일 세포 수준에서 프로파일링하려는 공동 노력의 결과물입니다. 전체 데이터셋에는 고처리량이지만 낮은 커버리지의 10X 데이터와 저처리량이지만 높은 커버리지의 Smartseq2 데이터가 모두 포함되어 있습니다.

이 워크숍에서는 마우스 뇌의 Smartseq2 데이터를 사용할 것입니다. 이 데이터는 다음으로 구성됩니다:  
1. 각 열이 유전자(또는 전사체)에 해당하고 각 행이 단일 세포에 해당하는 __발현 행렬__  
2. 각 세포를 설명하는 __메타데이터__ 표

## 데이터 다운로드

데이터는 이 강의 리포지토리의 `scRNA-python-workshop/content/data.zip`에 번들로 제공됩니다. [여기](https://github.com/chanzuckerberg/scRNA-python-workshop/blob/master/content/data.zip)에서 데이터 파일을 직접 찾을 수도 있습니다. 폴더의 압축을 풀어 파일이 `scRNA-python-workshop/content/data/` 디렉토리에 위치하도록 하십시오.

## 데이터 읽기

이제 쉼표로 구분된 파일에서 카운트 행렬을 읽어올 수 있습니다. 그런 다음 결과 데이터프레임을 검사합니다:

In [ ]:
import pandas as pd ## 데이터프레임 작업을 위한 pandas 라이브러리 로드

### 연습  

열 이름은 무엇을 나타냅니까? 행 이름은 무엇을 나타냅니까? 이 데이터셋에는 몇 개의 세포와 유전자가 있습니까?

힌트: 도움이 필요하십니까? `help(pd.DataFrame.shape)`를 시도해 보십시오.

<p>
<details>
<summary><h3>해결책</h3></summary>
열 이름은 유전자를 나타냅니다. 
행 이름은 데이터셋 작성자가 할당한 고유한 세포 식별자를 나타냅니다.
<p/>
<code>shape</code>를 요청하여 데이터셋에 몇 개의 유전자와 세포가 있는지 알아낼 수 있습니다:  <br>
<code>print(count_dataframe.shape)</code><br>
결과:<br>
<code>(3401, 23433)</code>
<p/>
이는 <code>(행 수, 열 수)</code>를 나타냅니다.
</details>
</p>

In [ ]:
help(pd.DataFrame.shape)

In [ ]:
metadata_dataframe = pd.read_csv()



## AnnData 객체 빌드

In [ ]:
import scanpy as sc # AnnData 데이터 구조가 어떻게 작동하는지 Python에 알려주는 scanpy 라이브러리 가져오기

In [ ]:
# help(sc.AnnData)
adata = sc.AnnData(X = count_dataframe, obs = metadata_dataframe)
print(adata)

### 스파이크인 라벨링
이것은 smartseq2 데이터이므로 스파이크인이 있을 수 있습니다. 이러한 유전자 이름은 `ERCC`로 시작합니다. `adata.var`에서 유전자 주석으로 라벨을 지정할 수 있습니다.

In [ ]:
is_spike_in = {}
number_of_spike_ins = 0

for gene_name in adata.var_names:
    if 'ERCC' in gene_name:
        is_spike_in[gene_name] = True # 스파이크인을 찾았음을 기록
        number_of_spike_ins += 1 # 카운터 증가
    else:
        is_spike_in[gene_name] = False # 스파이크인이 아님을 기록
        
adata.var['ERCC'] = pd.Series(is_spike_in) # adata.var의 인덱스와 is_spike_in의 키가 일치하므로 anndata가 일치하도록 처리합니다.
print('발견된 스파이크인 수: ', number_of_spike_ins)

이제 AnnData 객체 빌드를 마쳤으므로 나중에 사용하기 위해 다음과 같이 파일에 저장할 수 있습니다:

In [ ]:
adata.write('../data/brain_raw.h5ad') ## h5ad 확장자는 AnnData 전용입니다.